In [6]:
import json
import numpy as np
from bokeh.plotting import figure
from bokeh.io import output_notebook, show
output_notebook()

Loading BokehJS ...

In [7]:
colors = ['red', 'blue', 'purple', 'green', 'orange', 'black']

In [8]:
info = json.loads(open('runs/1/info.json').read())
p = figure(title='CNN - MNIST', plot_width=600, plot_height=400)

c = 0
for k in info.keys():
    if k.startswith('uncertainty'):
        xy = np.array(info[k])
        area = np.trapz(y=xy[:, 1], x=xy[:, 0])
        p.line(xy[:, 0], xy[:, 1], legend='{0}: {1:.6f}'.format(k, area), color=colors[c], line_width=3)
        c += 1

p.xaxis.axis_label = 'data proportion'
p.yaxis.axis_label = 'accuracy'
p.legend.location = 'bottom_left'

show(p)

In [9]:
info = json.loads(open('runs/2/info.json').read())
p = figure(title='CNN - MNIST', plot_width=600, plot_height=400)

c = 0
for k in info.keys():
    if k.startswith('uncertainty'):
        xy = np.array(info[k])
        area = np.trapz(y=xy[:, 1], x=xy[:, 0])
        p.line(xy[:, 0], xy[:, 1], legend='{0}: {1:.6f}'.format(k, area), color=colors[c], line_width=3)
        c += 1

p.xaxis.axis_label = 'data proportion'
p.yaxis.axis_label = 'accuracy'
p.legend.location = 'bottom_left'

show(p)

In [41]:
info = json.loads(open('runs/7/info.json').read())
p = figure(title='MLP - BOSTON', plot_width=800, plot_height=600)

c = 0
for k in info.keys():
    if k.startswith('uncertainty'):
        xy = np.array(info[k])
        area = np.trapz(y=xy[:, 1], x=xy[:, 0])
        p.line(xy[:, 0], xy[:, 1], legend='{0}: {1:.6f}'.format(k, area), color=colors[c], line_width=3)
        c += 1

p.xaxis.axis_label = 'data proportion'
p.yaxis.axis_label = 'RMSE'
p.legend.location = 'bottom_right'

show(p)